In [ ]:
##################################################################
# Rob Mowry, Heena Waichulis, Esther Lowe, and Kevin Stradinger  #
# Analysis of Chicago Crime                                      #
# 06/22/2019                                                     #
##################################################################

In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from sodapy import Socrata

In [4]:
#Creating base URL and inserting Key
copa = Socrata('data.cityofchicago.org', 'thHdbKC6b0gXkzsjHByVYU8qf')

In [ ]:
#Calling the COPA dataset from Chicago Open Data
copa_data = copa.get("mft5-nfa8", limit=81600)

In [ ]:
#Creating Dataframe, veiwing length & head
copa_df = pd.DataFrame.from_records(copa_data)
print(len(copa_df))
copa_df.head()

# What organizations are complaints being investigated by?
Complaints filed against CPD are investigated by one of three organizations
Bureau of Internal Affaairs: an internal organism of CPD intended to investigate offenses such as: Criminal misconduct, Operational violations, Theft of money or property, Planting of drugs, Substance abuse, Residency violations, and Medical roll abuse. COPA and its predeccesor IPRA: Both COPA and IPRA are independent of CPD and investigate what are generlly more major offenses including: Bias-based verbal abuse, Coercion, Death or serious bodily injury in custody, Domestic violence, Excessive force, Improper search and seizure, Firearm discharge, Taser discharge that results in death or serious bodily injury, Pattern or practices of misconduct, Unlawful denial or access to counsel.

In [ ]:
# Sorting complaints by investigating organization
complaint_assignments =copa_df['assignment'].value_counts()
assignment_labels = ['BIA','IPRA',"COPA"]
assignment_colors = ['White','skyblue','coral']
plt.title('Percent of Cases assigned by Oversight office')

plt.pie(complaint_assignments, labels = assignment_labels, colors = assignment_colors, shadow =True, startangle = 90, autopct='%.1f%%')

plt.savefig("Resources/Investigating organization Pie")

# What does this tell us?
Just over 71 percent of complaints made against CPD are handled internally by BIA. These cases are handled internally and as such generally stay out of public view. This is evident from the head of our data, where we see that every case assigned to BIA has no more than a complaint date and a log number. While it is possible to get more information on these cases individually, it isn't feasible to do so on the scale which we are working at. While no abuse of power by those tasked with law enforcment should be considered minor the cases that fall under BIA jurisdiction tend to be less severe than those assigned to IPRA and COPA. The significant gap in cases assigned to COPA and IPRA can be explained by COPA's relative youth. While both organizations operated at the same time for a short period COPA has since replaced IPRA completely. An increase in scrutiny of police abuses of power came following the release of a dashcam footage of the shooting of LaQuan McDonald in 2014. IPRA came under fire not just specifically for thatcase but for failing to keep police accountable and was later replaced by COPA.

In [ ]:
# Remove BIA complaints from the set, to analyze IPRA and COPA
copa_df = copa_df[copa_df.assignment != 'BIA']

In [ ]:
complaint_assignments_lessBIA =copa_df['assignment'].value_counts()
complaint_assignments_lessBIA

In [ ]:
copa_df['current_category'].value_counts()

In [ ]:
excessive_force = len(copa_df[copa_df["current_category"] == "Excessive Force"])
taser_notification = len(copa_df[copa_df["current_category"] == "Taser Notification"])
miscellaneous = len(copa_df[copa_df["current_category"] == "Miscellaneous"])
verbal_abuse = len(copa_df[copa_df["current_category"] == "Verbal Abuse"])
unnecessary_display_of_weapon = len(copa_df[copa_df["current_category"] == "Unnecessary Display of Weapon"])
firearm_discharge_at_animal = len(copa_df[copa_df["current_category"] == "Firearm Discharge at Animal"])
civil_suits = len(copa_df[copa_df["current_category"] == "Civil Suits"])
domestic_violence = len(copa_df[copa_df["current_category"] == "Domestic Violence"])
search_or_seizure = len(copa_df[copa_df["current_category"] == "Search or Seizure"])
firearm_discharge_hits = len(copa_df[copa_df["current_category"] == "Firearm Discharge - Hits"])
death_or_injury_in_custody = len(copa_df[copa_df["current_category"] == "Death or Injury In Custody"])
oc_discharge = len(copa_df[copa_df["current_category"] == "OC Discharge"])
firearm_discharge_no_hits = len(copa_df[copa_df["current_category"] == "Firearm Discharge - No Hits"])
coercion = len(copa_df[copa_df["current_category"] == "Coercion"])
operational_violation = len(copa_df[copa_df["current_category"] == "Operational Violation"])
taser_discharge = len(copa_df[copa_df["current_category"] == "Taser Discharge"])
motor_vehicle_death = len(copa_df[copa_df["current_category"] == "Motor Vehicle Related Death"])
legal_violation = len(copa_df[copa_df["current_category"] == "Legal Violation"])
bias = len(copa_df[copa_df["current_category"] == "Bias"])
unlawful_denial_of_counsel = len(copa_df[copa_df["current_category"] == "Unlawful Denial of Counsel"])

In [ ]:
# All complaints by category
complaint_df=({'Excessive Force': excessive_force,
              'Taser Notification': taser_notification,
              'Miscellaneous':miscellaneous,
              'Verbal Abuse':verbal_abuse,
              'Unnecessary Display of Weapon':unnecessary_display_of_weapon,
              'Firearm Discharge at Animal':firearm_discharge_at_animal,
              'Civil Suit': civil_suits,
              'Domestic Violence': domestic_violence,
              'Search or Seizure': search_or_seizure,
              'Firearm Discharge -- Hits':firearm_discharge_hits,
              'Death or Injusry in Custody':death_or_injury_in_custody,
              'OC Discharge(Chemical Agent)': oc_discharge,
              'Firearm Dischagre -- No Hits': firearm_discharge_no_hits,
              'Coercion': coercion,
              'Operational Violation': operational_violation,
              'Taser Discharge': taser_discharge,
              'Motor Vehicle Death': motor_vehicle_death,
              'Legal Violation': legal_violation,
              'Bias': bias,
              'Unlawful Denial of counsel':unlawful_denial_of_counsel})
complaint_df = pd.DataFrame(complaint_df, index=[0])
complaint_df=complaint_df.rename(columns ={0: "Complaint Count"}) 
complaint_df=complaint_df.transpose()
complaint_df

In [ ]:
# Complaint counts Bar Graph
complaint_bar = complaint_df.iloc[0].plot.bar()
plt.xlabel('Complaint Type')
plt.ylabel('Complaint Count')
plt.title('Counts for COPA and IPRA Complaint Types')

plt.savefig("Resources/Complaint types Bar")

In [ ]:
#violent complaints
violent_complaint_df=({'Excessive Force': excessive_force,
              'Taser Notification': taser_notification,
              'Firearm Discharge at Animal':firearm_discharge_at_animal,
              'Domestic Violence': domestic_violence,
              'Firearm Discharge -- Hits':firearm_discharge_hits,
              'Death or Injusry in Custody':death_or_injury_in_custody,
              'OC Discharge(Chemical Agent)': oc_discharge,
              'Firearm Dischagre -- No Hits': firearm_discharge_no_hits,
              'Taser Discharge': taser_discharge,
              'Motor Vehicle Death': motor_vehicle_death})
violent_complaint_df = pd.DataFrame(violent_complaint_df, index=[0])
violent_complaint_df.transpose()

In [ ]:
# Bar graph for violent complaints
Violent_complaint_bar = violent_complaint_df.iloc[0].plot.bar()
plt.xlabel('Complaint Type')
plt.ylabel('Complaint Count')
plt.title('Counts for COPA and IPRA Complaint Types (Violent Complaints Only)')

plt.savefig("Resources/Violent complaint types Bar")

# Are Violent cases more likely to be reported?
We can't determine whether or not violent cases are more likely to be reported, because we obviously don't have any information on cases that aren't reported. However, we can observe that of the complaints those whose category is inherently violent make up 69.5% of all complaints handled by COPA and IPRA. This also discounts every complaints like miscellaneous and coercion, which while not inherently violent certainly could be.

While we can't ultimately determine if violent cases are more likely to be reported we can determine that case reported are more likely than not violent.

# What are the outcomes of these investigation?
Investigation findings are divided into one of six categories in the dataset
No Finding: The investigation is either waiting to begin, still underway or has been closed No Affidavit: According to COPA's website state law stipulates that "in most instances, an affidavit be signed where an allegation of misconduct is made against a police officer." In certain cases Copa will pursue an affadavit override to pursue the case but if they cannot aquire an affadavit or and override the case will be closed. Not Sustained: The allegation is not supported by sufficient evidence which could be used to prove or disprove the allegation. Sustained: The allegation was supported by sufficient evidence to justify disciplinary action. Recommendations of disciplinary action may range from violation noted to separation from the Department. Unfounded: The complaint was not based on facts as shown by the investigation, or the reported incident did not occur. Exonerated: The incident occurred, but the action taken by the officer(s) was deemed lawful and proper.

In [ ]:
# variable for investigation outcome
investigation_finding=copa_df['finding_code'].value_counts()
investigation_finding

In [ ]:
no_finding = len(copa_df[copa_df["finding_code"] == "No Finding"])
no_affadavit = len(copa_df[copa_df["finding_code"] == "NO AFFIDAVIT"])
not_sustained = len(copa_df[copa_df["finding_code"] == "NOT SUSTAINED"])
unfounded = len(copa_df[copa_df["finding_code"] == "UNFOUNDED"])
sustained = len(copa_df[copa_df["finding_code"] == "SUSTAINED"])
exonerated = len(copa_df[copa_df["finding_code"] == "EXONERATED"])

In [ ]:
finding_df = ({'No Finding': no_finding,
              'No Affadavit': no_affadavit,
              'Not Sustained': not_sustained,
              'Unfounded': unfounded,
              'Sustained':sustained,
              'Exonerated': exonerated
              })
finding_df = pd.DataFrame(finding_df, index=[0])
finding_df.transpose()

In [ ]:
# Finding DF
investigation_status = finding_df.iloc[0].plot.bar()
plt.xlabel('Investigation Status or Outcome')
plt.ylabel('Investigation status or Outcome Count')
plt.title('Count for all investigations Outcome and Status')

plt.savefig("Resources/All Investigation Findings Bar")

Are cases handled by COPA more likely to be sustained than those handled by IPRA?
Since COPA was formed due to the failure of IPRA to maintain accountabilty, I would expect that cases handled by COPA would be more likely to result in some sort of discipline than those handled by IPRA.

In [ ]:
# New DF containing only IPRA cases
ipra_df = copa_df[copa_df.assignment == 'IPRA']
ipra_df.head()

In [ ]:
# IPRA investigation findings
ipra_investigation_finding=ipra_df['finding_code'].value_counts()
ipra_investigation_finding

ipra_no_finding = len(ipra_df[ipra_df["finding_code"] == "No Finding"])
ipra_no_affadavit = len(ipra_df[ipra_df["finding_code"] == "NO AFFIDAVIT"])
ipra_not_sustained = len(ipra_df[ipra_df["finding_code"] == "NOT SUSTAINED"])
ipra_unfounded = len(ipra_df[ipra_df["finding_code"] == "UNFOUNDED"])
ipra_sustained = len(ipra_df[ipra_df["finding_code"] == "SUSTAINED"])
ipra_exonerated = len(ipra_df[ipra_df["finding_code"] == "EXONERATED"])

In [ ]:
ipra_finding_df = ({'No Finding': ipra_no_finding,
              'No Affadavit': ipra_no_affadavit,
              'Not Sustained': ipra_not_sustained,
              'Unfounded': ipra_unfounded,
              'Sustained':ipra_sustained,
              'Exonerated': ipra_exonerated
              })
ipra_finding_df = pd.DataFrame(ipra_finding_df, index=[0])
ipra_finding_df.transpose()

In [ ]:
# IPRA finding DF
ipra_investigation_status = ipra_finding_df.iloc[0].plot.bar()
plt.xlabel('Investigation Status or Outcome')
plt.ylabel('Investigation status or Outcome Count')
plt.title('Count for IPRA investigation Outcome and Status')

plt.savefig("Resources/IPRA Investigation Findings Bar")

# Sustained, cases in which some officer discipline occured make up just 4.3% of cases handled by IPRA

In [ ]:
# New DF containing only COPA cases
copa_only_df = copa_df[copa_df.assignment == 'COPA']
copa_only_df.head()

In [ ]:
#COPA investigation findings
copa_investigation_finding=copa_only_df['finding_code'].value_counts()
copa_investigation_finding

In [ ]:
# Counts for investigation findings
copa_no_finding = len(copa_only_df[copa_only_df["finding_code"] == "No Finding"])
copa_no_affadavit = len(copa_only_df[copa_only_df["finding_code"] == "NO AFFIDAVIT"])
copa_not_sustained = len(copa_only_df[copa_only_df["finding_code"] == "NOT SUSTAINED"])
copa_unfounded = len(copa_only_df[copa_only_df["finding_code"] == "UNFOUNDED"])
copa_sustained = len(copa_only_df[copa_only_df["finding_code"] == "SUSTAINED"])
copa_exonerated = len(copa_only_df[copa_only_df["finding_code"] == "EXONERATED"])

In [ ]:
copa_finding_df = ({'No Finding': copa_no_finding,
              'No Affadavit': copa_no_affadavit,
              'Not Sustained': copa_not_sustained,
              'Unfounded': copa_unfounded,
              'Sustained':copa_sustained,
              'Exonerated': copa_exonerated
              })
copa_finding_df = pd.DataFrame(copa_finding_df, index=[0])
copa_finding_df.transpose()

In [ ]:
# COPA finding DF
count_labels=['283','253','35','102','52','24']
copa_investigation_status = copa_finding_df.iloc[0].plot.bar(tick_label= count_labels)
plt.xlabel('Investigation Status or Outcome')
plt.ylabel('Investigation status or Outcome Count')
plt.title('Count for COPA investigation Outcome and Status')


plt.savefig("Resources/COPA Investigation Findings Bar")

IPRA, largely considered to be failing it mandate to investigate complaints to ensure police accountability found that only 4.3% of complaints resulted in a sustained verdit, meaning the officer was found to be acting outside of their powers, resulting in discipline of some manor. COPA the successor to IPRA after its failure has reached sustained verdicts in 6.9% of its cases. It is important to note that the sample sizes vary due to COPA's relative youth, with COPA cases making up only about 6.8% of cases given to one of the two independent review organiztions. In addition the cases are obviously different and as such are hard to compare exactly. With those limitations accounted for, COPA is a more effective organiztion when measuring the prectentage of cases sustained.